# Reflexive Verbs

Gotta love the lack of documentation about obscure languages. Lets scrape the conjugation rules from the internet.

## First scrape a bunch of reflexive verb tables

In [50]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint

def get_soup(suffix, base_url='https://en.wiktionary.org'):
    return BeautifulSoup(requests.get(base_url + suffix).text, 'lxml')

words = {}

base_soup = get_soup('/wiki/Category:Latvian_reflexive_verbs')
for word_soup in base_soup.find_all('div', class_='mw-content-ltr')[0].find_all('a', href=True):
    if word_soup.has_attr('title'):
        conjug_soup = get_soup(word_soup['href']).find_all('table', class_='inflection-table')
        data = [
            [
                {"string":ele.text.strip(),
                 "type": "th",
                 "rowspan": int(ele['rowspan']) if ele.has_attr('rowspan') else 1,
                 "colspan": int(ele['colspan']) if ele.has_attr('colspan') else 1}
                for ele in row.find_all('th')
                #if ele.text.strip()
            ] + [
                {"string":ele.text.strip(),
                 "type": "td",
                 "rowspan": int(ele['rowspan']) if ele.has_attr('rowspan') else 1,
                 "colspan": int(ele['colspan']) if ele.has_attr('colspan') else 1}
                for ele in row.find_all('td')
                #if ele.text.strip()
            ]
            for row in conjug_soup[0].find_all('tr')
        ]
        words[word_soup.text] = data
#pprint(words)

## Try to extract out how the stem is used

In [49]:
import matplotlib.pyplot as plt
import numpy as np

for word in words:
    conjugated_forms = ''.join([
        col['string']
        for row in words[word]
        for col in row
        if col['type'] == 'td'
    ])
    
    stem_length_weight = 2.5
    occurances = np.array([
        conjugated_forms.count(word[:i]) * (i+1)**stem_length_weight
        for i in range(len(word))
    ])
    print(word[:np.argmax(occurances)], word[np.argmax(occurances):])

apcep ties
apsprie sties
atcer ēties
atvaino ties
baid īties
bī ties
cen sties
cepinā ties
cep ties
ciemo ties
do ties
izcep ties
izskat īties
klaus īties
kust ēties
lik ties
mērcē ties
nocep ties
pacep ties
paskat īties
prostituē ties
pārcep ties
redz ēties
rie ties
sacep ties
sapulcē ties
sauļo ties
skat īties
smie ties
uzcep ties
uzdo ties
vaislo ties
vel ties
vēlē ties


## Now we can display our stem table

In [37]:
from IPython.core.display import display, HTML

def make_html(table_json):
    soup = BeautifulSoup('<table></table>', 'lxml')
    table = soup.table
    for row in table_json:
        tr = soup.new_tag('tr')
        for col in row:
            td = soup.new_tag(col['type'], **col)
            td.string = col['string']
            tr.append(td)
        table.append(tr)
    return str(soup)
    
display(HTML(make_html(words['apcepties'])))